In [98]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json

In [99]:
#  Bluetooth bug circumnavigate
options = webdriver.ChromeOptions() 
options.add_experimental_option("excludeSwitches", ["enable-logging"])



In [100]:
ScrollNumber = 2  # The depth we wish to load
sleepTimer = 2    # Waiting 1 second for page to load
pageNumber = 3

searchList = ["cow", "rabbit", "lamb", "cat", "monkey", "fish", "dog", "horse", "bird"]
dicoLink = {clef : [] for clef in searchList}

Le code principal qui ouvre un driver chrome et effectue les recherches sur internet pour récupérer les images : 

In [101]:
driver = webdriver.Chrome(options=options)  # path=r'to/chromedriver.exe'

for var in searchList:
    #  Any infinity scroll URL

    for page in range(pageNumber):
        url = "https://fr.freepik.com/photos-vecteurs-libre/coloriages-imprimer-" + var + "/" + str(page+1)
        driver.get(url)

        for _ in range(0,ScrollNumber):
            driver.execute_script("window.scrollTo(1,5000)")
            print("scrolling")
            time.sleep(sleepTimer)

        soup = BeautifulSoup(driver.page_source,'html.parser')

        for link in soup.find_all('img'):
            linkImage = link.get('src')
            dicoLink[var].append(linkImage)

print(dicoLink)
with open('dictionnaireLienImagesAnimaux.json', 'w') as fichier:
    json.dump(dicoLink, fichier)

scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
scrolling
{'cow': ['https://fps.cdnpk.net/menu-top/new-collections/IG-vectors.webp', 'https://fps.cdnpk.net/menu-top/new-collections/IG-photos.webp', 'https://fps.cdnpk.net/menu-top/new-collections/Icon-generator.webp', 'https://fps.cdnpk.net/menu-top/new-collections/video-generator.webp', 'https://fps.cdnpk.net/menu-top/new-collections/Designer.webp', 'https://fps.cdnpk.net/menu-top/new-collections/mockup-generator.webp', 'https://img.freepik.com/vecteurs-libre/ill

Il faut maintenant filtrer les liens obtenus : on ne garde que ceux qui contiennent le mot cléf vecteurs :

In [102]:

def isAbout(contextList, irrelevantList, url):                            #pas sûr de comment faire les choses bien pour le moment mais on garde l'idée
    boolRet = False
    for element in contextList:
        if(element in url):
            print(element in url, " : ", element, "  : ", url)
            boolRet= True
    for element in irrelevantList:
        if(element in url):
            print(element in url, " : ", element, "  : ", url)
            boolRet= False   
    if 'jpg' not in url or 'img' not in url:
        boolRet = False
    return boolRet

In [103]:
with open('dictionnaireLienImagesAnimaux.json', 'r') as fichier:
    dicoLink = json.load(fichier)

relevantList = ["cow", "rabbit", "lamb", "cat", "monkey", "fish", "dog", "horse",  "bird", "vache", "lapin", "mouton", "agneau", "chat", "chien", "poisson", "cheval", "oiseau", "singe"]
irrelevantList = ["twitter", "analytics", "Twitter"]
dicoLinkClean = {}
for clef, valeur in dicoLink.items():
    valeur = valeur[6:-5]                           #enlève les images téléchargées sur Freepick inutiles (logos, image de base, ...) 
    valeurClean = [linkClean for linkClean in valeur if ('vecteurs' in linkClean and isAbout(relevantList, irrelevantList, linkClean))]
    dicoLinkClean[clef] = valeurClean

print(dicoLinkClean)

True  :  vache   :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149309137.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149291238.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149296563.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-libre/contour-vache-design-plat-dessine-main_23-2149261305.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149291237.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-premium/coloriage-vache-dessin-pour-enfants_925324-4159.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-premium/illustration-du-contour-vache-dessinee-main_1208459-189.jpg?semt=ais_hybrid
True  :  vache   :  https://img.freepik.com/vecteurs-premium/mere-vach

On peut maintenant télécharger et enregistrer des des dossiers adaptés les images :

In [104]:
import requests
import os

def telecharger_image(url, chemin_fichier):
    try:
        # Envoyer une requête GET à l'URL
        reponse = requests.get(url, stream=True)
        reponse.raise_for_status()  # Vérifie si la requête a réussi

        # Ouvrir un fichier en mode binaire pour écrire l'image
        with open(chemin_fichier, 'wb') as fichier:
            for chunk in reponse.iter_content(chunk_size=8192):
                fichier.write(chunk)

        print(f"Image téléchargée avec succès et enregistrée sous : {chemin_fichier}", " url : ", url)

    except requests.exceptions.HTTPError as http_err:
        print(f"Erreur HTTP : {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Erreur de connexion : {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Erreur de délai d'attente : {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Erreur lors de la requête : {req_err}")
    except Exception as err:
        print(f"Une erreur est survenue : {err}")

In [105]:
for animals in searchList:
    for i, url in enumerate(dicoLinkClean[animals]):
        pathFile = "D:\scrapping\\"+ animals 
        if not os.path.exists(pathFile):
            os.makedirs(pathFile)
            print(f"Dossier créé : {pathFile}")
        pathFile += "\\" + animals
        pathFile += str(i)
        pathFile += ".jpg"
        telecharger_image(url, pathFile)

Dossier créé : D:\scrapping\cow
Image téléchargée avec succès et enregistrée sous : D:\scrapping\cow\cow0.jpg  url :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149309137.jpg?semt=ais_hybrid
Image téléchargée avec succès et enregistrée sous : D:\scrapping\cow\cow1.jpg  url :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149291238.jpg?semt=ais_hybrid
Image téléchargée avec succès et enregistrée sous : D:\scrapping\cow\cow2.jpg  url :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-2149296563.jpg?semt=ais_hybrid
Image téléchargée avec succès et enregistrée sous : D:\scrapping\cow\cow3.jpg  url :  https://img.freepik.com/vecteurs-libre/contour-vache-design-plat-dessine-main_23-2149261305.jpg?semt=ais_hybrid
Image téléchargée avec succès et enregistrée sous : D:\scrapping\cow\cow4.jpg  url :  https://img.freepik.com/vecteurs-libre/illustration-contour-vache-dessine-main_23-21492

In [ ]:
from PIL import Image

def redimensionner_images(dossier_source, dossier_destination, taille_cible):
    # Créer le dossier de destination s'il n'existe pas
    if not os.path.exists(dossier_destination):
        os.makedirs(dossier_destination)

    # Parcourir toutes les images dans le dossier source
    for nom_fichier in os.listdir(dossier_source):
        chemin_image = os.path.join(dossier_source, nom_fichier)
        
        try:
            # Charger l'image
            with Image.open(chemin_image) as img:
                # Redimensionner l'image
                img_resized = img.resize(taille_cible, Image.ANTIALIAS)
                
                # Enregistrer dans le dossier de destination
                chemin_sauvegarde = os.path.join(dossier_destination, nom_fichier)
                img_resized.save(chemin_sauvegarde)

                print(f"Image {nom_fichier} redimensionnée et enregistrée.")
        except Exception as e:
            print(f"Erreur avec l'image {nom_fichier}: {e}")

# Exemple d'utilisation
dossier_source = "images_originales"  # Dossier contenant les images
dossier_destination = "images_redimensionnees"  # Dossier pour enregistrer les images redimensionnées
taille_cible = (128, 128)  # Taille cible (largeur, hauteur)

redimensionner_images(dossier_source, dossier_destination, taille_cible)